In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploration of relationship between dissolved oxygen (DO or O2) readings and packaged beer volume using test dataset of 3 different lots of beers from Siboire craft brewery. 
---------
## The beers :

- [Notre Folie](https://siboire.ca/fr/boutique/bieres/3968024-notre-folie-100-qc/)
- [IPA Test #96](https://siboire.ca/fr/boutique/bieres/4847087-ipa-test-96-citra-spectrum-citra-lupomax-citra/)
- [Inspiration](https://siboire.ca/fr/boutique/bieres/213673-inspiration/)

## The measurements :
1. Weight of packaged beer : 
    Lab balance with 0.01 g readability
2. 3-minutes shaken CO2 and O2 readings of packaged using CBoxQC (Anton Paar; [CboxQC](https://www.anton-paar.com/us-en/products/details/cboxqc/)) and c-DGM (Haffmans; [c-DGM](https://foodandbeverage.pentair.com/en/products/haffmans-portable-optical-co2-o2-tpo-meter-c-dgm))

---

## Useful data and formulas :
- Empty can weight (hard print) = 15.8 g
- density p = mass (g) / volume (mL)
- Plato to SG conversion with ASBC's online conversion tools : 
[Extract to Specific Gravity](https://www.asbcnet.org/Methods/Tools/extras/Documents/Extract_to_Specific_Gravity.pdf)
- Water density at 20C from Handbook of Chemistry : [H20_20C_g/mL](https://collett.atmos.colostate.edu/classes/AT560/2007/water-density.pdf)

--- 

## Question
<code style="background:yellow;color:red">**Is there relationship between dissolved O2 levels and volume of beer in final package?**</code>
***

# Import raw data and cleanup
***
Import data as df using the Pandas lib


In [13]:
# Import raw data + check sample pop and NAs
rawDf = pd.read_csv("../input/vol-o2-3beers-1668-1671-1672/o2-vol-3beers-1668-1670-1671_full.csv", sep=";", decimal=",")
rawDf.head(10)


In [14]:
# Looking for NAs
rawDf.count()
# Dropping NAs we have n=28 samples left

In [15]:
# change weight column name + remove NAs
rawDf = rawDf.rename(columns={"weight (g)": "full weight (g)"})
df = rawDf.dropna()


In [16]:
# Check if dtypes OK
df.dtypes


# Calculate the volume of beer in package
***
## Strategy :
1. Determine weight of liquid beer in can
2. Calculate specific gravity (SG)
3. Calculate the volume of beer in can (mL)


In [17]:
# Determine weight of beer liquid only
# Regular hard printed can body + lid weight = 15.8 g
emptyCan = float(input("Enter weight of empty can + lid :"))
df["liquid weight (g)"] = df["full weight (g)"] - emptyCan

# Convert Plato density to specific gravity (SG)
df["SG"] = df["Density (Plato)"] / (258.6-((df["Density (Plato)"]/258.2)*227.1))+1

# Calculate beer liquid volume in package (mL)
watDens20C = 0.998207
df["beer volume (mL)"] = df["liquid weight (g)"] / (df["SG"]*watDens20C)
df.head()


# Exploratory data analysis
***
## Data visualization + cleaning
- Check distributions for beer volume, CO2 and O2
- Remove outliers

In [18]:
# beer volume histogram
sns.histplot(data=df, x="beer volume (mL)")

In [19]:
# O2 histogram
sns.histplot(data=df, x="O2 (ppb)")

In [20]:
# CO2 histo
sns.histplot(data=df, x="CO2 (vol/vol)", kde=True)

In [ ]:
# Remove outliers
